In [1]:
import pandas as pd

In [8]:
df = pd.read_csv('rule_dataset.csv')

def clean_braces(x):
    x = x.replace("()", "").replace(",", "").replace(" ", "")
    return str(x)
def add_braces(x):
    x = x+"()"
    return x

df['Syscall'] = df['Syscall'].apply(clean_braces)
df.head(3)

# df['Syscall'] = df['Syscall'].apply(add_braces)
# df.to_csv("rule_dataset.csv", index=False)

,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,FILE,FILE_READ,man,read,read,read from a file descriptor
1,FILE,FILE_READ,man,readlink,read,read value of a symbolic link
2,FILE,FILE_READ,man,readlink,place,readlink() places the contents of the symbolic...


- 印出各類別的個數和內容

In [22]:
entityType_lst = df['EntityType'].unique()
lenSyscall = len(df['Syscall'].unique())
lenEnVerb = len(df['EnVerb'].unique())
print(f"For total {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs\n")

for entityType in entityType_lst:
    dataFileType = df[df['EntityType'] == entityType]
    lenEnTp = len(dataFileType)
    lenSyscall = len(dataFileType['Syscall'].unique())
    lenEnVerb = len(dataFileType['EnVerb'].unique())
    actionType_lst = dataFileType['ActionType'].unique()

    print(f"EntityType: {entityType:<6} has {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs & {len(actionType_lst):2} actionType")

    for actionType in actionType_lst:
        dataActionType = dataFileType[dataFileType['ActionType'] == actionType]
        lenSyscall = len(dataActionType['Syscall'].unique())
        lenEnVerb = len(dataActionType['EnVerb'].unique())

        print(f"\t{actionType:<12} has {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs")

    print()

    

For total 88 uq syscalls & 43 uq verbs

EntityType: FILE   has 38 uq syscalls & 25 uq verbs & 12 actionType
	FILE_READ    has  2 uq syscalls &  2 uq verbs
	ACCESS       has  1 uq syscalls &  2 uq verbs
	STATUS       has  6 uq syscalls &  1 uq verbs
	DIR          has  2 uq syscalls &  1 uq verbs
	WAIT         has  2 uq syscalls &  1 uq verbs
	FD_DUP       has  2 uq syscalls &  3 uq verbs
	FD_OTHER     has  1 uq syscalls &  2 uq verbs
	CREATE_OPEN  has 10 uq syscalls &  6 uq verbs
	UPDATE       has  5 uq syscalls &  3 uq verbs
	DELETE       has  5 uq syscalls &  3 uq verbs
	CLOSE        has  1 uq syscalls &  2 uq verbs
	OTHER        has  1 uq syscalls &  1 uq verbs

EntityType: PROC   has 21 uq syscalls & 16 uq verbs &  6 actionType
	PROC_EXE     has  4 uq syscalls &  2 uq verbs
	PROC_KILL    has  2 uq syscalls &  2 uq verbs
	PROC_TRACE   has  1 uq syscalls &  5 uq verbs
	PROC_MEM     has  3 uq syscalls &  2 uq verbs
	PROC_WAIT    has  2 uq syscalls &  1 uq verbs
	PROC_OTHER   has  9 uq 

試寫 rule
- input: object 和 entityType
- output: 對應的 syscall (1或多個)

In [ ]:
! pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
